In [13]:
!nvidia-smi

Wed Jul  5 22:01:32 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 536.40                 Driver Version: 536.40       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3060 Ti   WDDM  | 00000000:01:00.0  On |                  N/A |
|  0%   46C    P2              61W / 225W |   1194MiB /  8192MiB |      4%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

计算设备

In [2]:
import torch
from torch import nn

torch.device('cpu'),torch.cuda.device('cuda'),torch.cuda.device('cuda:0'),

(device(type='cpu'),
 <torch.cuda.device at 0x204b8649490>)

查询可用gpu数量

In [3]:
torch.cuda.device_count()

1

In [4]:
def try_gpu(i=0):  #@save
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():  #@save
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]"""
    devices = [torch.device(f'cuda:{i}')
             for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

查询张量所在设备

In [20]:
x = torch.tensor([1,2,3])
x.device

device(type='cpu')

存储在GPU

In [27]:
X = torch.tensor([1,2,3],device = try_gpu(),dtype=torch.float32)#
X

tensor([1., 2., 3.], device='cuda:0')

In [10]:
Y = torch.rand(2, 3, device=try_gpu(1))#没有第二块gpu，放在了cpu上
Y

tensor([[0.5562, 0.0859, 0.4489],
        [0.9830, 0.9834, 0.2425]])

要计算x+y，我们需要决定在哪里执行

In [16]:
Z = Y.cuda(0)
print(Y)
print(Z)

tensor([[0.5562, 0.0859, 0.4489],
        [0.9830, 0.9834, 0.2425]])
tensor([[0.5562, 0.0859, 0.4489],
        [0.9830, 0.9834, 0.2425]], device='cuda:0')


数据都在同一块CPU上，就可以进行相加

In [17]:
Z+X

tensor([[1.5562, 2.0859, 3.4489],
        [1.9830, 2.9834, 3.2425]], device='cuda:0')

神经网络与GPU

In [28]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())

net(X)

tensor([-2.2752], device='cuda:0', grad_fn=<AddBackward0>)

确认模型和参数在同一个CPU

In [29]:
net[0].weight.data

tensor([[-0.4690,  0.1198, -0.5633]], device='cuda:0')